In [3]:
!pip install torch torchvision tqdm
import torch, os
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

    100% |████████████████████████████████| 519.5MB 31kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5968e000 @  0x7fb3ed1f31c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 1.7MB/s 
    100% |████████████████████████████████| 51kB 11.7MB/s 
    100% |████████████████████████████████| 2.0MB 2.9MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [4]:
transform = transforms.Compose([transforms.ToTensor(),])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [0]:
num_epochs = 15
learning_rate = 0.001

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
net = Net()

# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

Using GPU


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

In [0]:
def train(epoch, trainloader, optimizer, criterion):
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader), 0):
        # get the inputs
        inputs, labels = data
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print('epoch %d training loss: %.3f' %
            (epoch + 1, running_loss / (len(trainloader))))

In [0]:
def test(testloader, model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
                                    100 * correct / total))

In [0]:
def classwise_test(testloader, model):
########################################################################
# class-wise accuracy

    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))


In [14]:
print('Start Training')
os.makedirs('./models', exist_ok=True)

Start Training


In [15]:
for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

  0%|          | 0/12500 [00:00<?, ?it/s]

epoch  1


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 training loss: 1.888


  0%|          | 0/2500 [00:00<?, ?it/s]

...

epoch 15 training loss: 0.888


  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 59 %


100%|██████████| 2500/2500 [00:14<00:00, 172.05it/s]

Accuracy of plane : 58 %
Accuracy of   car : 68 %
Accuracy of  bird : 52 %
Accuracy of   cat : 48 %
Accuracy of  deer : 49 %
Accuracy of   dog : 40 %
Accuracy of  frog : 70 %
Accuracy of horse : 64 %
Accuracy of  ship : 71 %
Accuracy of truck : 70 %
Finished Training


In [25]:
num_epochs = 1
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3)#32-3+1=30
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)#15
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=4)#12
        self.fc1 = nn.Linear(in_features=16 * 6 * 6, out_features=480)
        self.fc2 = nn.Linear(in_features=480, out_features=360)
        self.fc3 = nn.Linear(in_features=360, out_features=120)
        self.fc4 = nn.Linear(in_features=120, out_features=84)
        self.fc5 = nn.Linear(in_features=84, out_features=10)


    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))#15
        x = self.pool2(F.relu(self.conv2(x)))#6
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 training loss: 2.206


  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 28 %


100%|██████████| 2500/2500 [00:16<00:00, 149.17it/s]

Accuracy of plane : 40 %
Accuracy of   car : 23 %
Accuracy of  bird : 16 %
Accuracy of   cat :  2 %
Accuracy of  deer :  1 %
Accuracy of   dog :  9 %
Accuracy of  frog : 81 %
Accuracy of horse : 17 %
Accuracy of  ship : 38 %
Accuracy of truck : 54 %
Finished Training


In [29]:
num_epochs = 1
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=1)#32-1+1=32
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)#16
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=2)#15
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=1)#14
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=28, kernel_size=5)#10
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)#5
        self.fc1 = nn.Linear(in_features=28*5*5, out_features=480)#1008
        self.fc2 = nn.Linear(in_features=480, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)


    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 28*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 training loss: 1.940


  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 41 %


100%|██████████| 2500/2500 [00:15<00:00, 164.09it/s]

Accuracy of plane : 40 %
Accuracy of   car : 60 %
Accuracy of  bird : 17 %
Accuracy of   cat : 11 %
Accuracy of  deer : 39 %
Accuracy of   dog : 39 %
Accuracy of  frog : 52 %
Accuracy of horse : 61 %
Accuracy of  ship : 41 %
Accuracy of truck : 51 %
Finished Training


In [30]:
num_epochs = 10
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=1)#32-1+1=32
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)#16
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=2)#15
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=1)#14
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=28, kernel_size=5)#10
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)#5
        self.fc1 = nn.Linear(in_features=28*5*5, out_features=480)#1008
        self.fc2 = nn.Linear(in_features=480, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)


    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 28*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 training loss: 1.970


  0%|          | 0/2500 [00:00<?, ?it/s]

...

epoch 10 training loss: 0.649


  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 62 %


100%|██████████| 2500/2500 [00:14<00:00, 168.10it/s]

Accuracy of plane : 70 %
Accuracy of   car : 90 %
Accuracy of  bird : 56 %
Accuracy of   cat : 35 %
Accuracy of  deer : 64 %
Accuracy of   dog : 48 %
Accuracy of  frog : 76 %
Accuracy of horse : 58 %
Accuracy of  ship : 78 %
Accuracy of truck : 50 %
Finished Training


In [34]:
num_epochs = 5
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=1)#32-1+1=32
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)#16
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=2)#15
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=1)#14
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=28, kernel_size=5)#10
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)#5
        self.fc1 = nn.Linear(in_features=28*5*5, out_features=480)#1008
        self.fc2 = nn.Linear(in_features=480, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=10)
        self.drop = nn.Dropout(p=0.4)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.drop(x)
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, 28*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')


  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1



  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 training loss: 2.046



  0%|          | 0/2500 [00:00<?, ?it/s]

...

epoch 5 training loss: 1.182



  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 58 %



100%|██████████| 2500/2500 [00:17<00:00, 146.69it/s]

Accuracy of plane : 64 %
Accuracy of   car : 73 %
Accuracy of  bird : 49 %
Accuracy of   cat : 29 %
Accuracy of  deer : 52 %
Accuracy of   dog : 54 %
Accuracy of  frog : 64 %
Accuracy of horse : 63 %
Accuracy of  ship : 71 %
Accuracy of truck : 66 %
Finished Training
